In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,41207
2,Huelva,Confirmados PDIA 14 días,3141
3,Huelva,Tasa PDIA 14 días,"612,0778689323226"
4,Huelva,Confirmados PDIA 7 días,1428
5,Huelva,Tasa PDIA 7 dias,"278,27035875051155"
6,Huelva,Total Confirmados,41406
7,Huelva,Curados,35134
8,Huelva,Fallecidos,396


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  41207.0


/tmp/ipykernel_3258/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  11493.0


/tmp/ipykernel_3258/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

/tmp/ipykernel_3258/2690025379.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_3258/2690025379.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_3258/2690025379.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 378 personas en los últimos 7 días 

Un positivo PCR cada 166 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,41207.0,1428.0,3141.0,513170.0,278.270359,612.077869,276.0
Huelva-Costa,24180.0,812.0,1830.0,289548.0,280.437095,632.019562,164.0
Condado-Campiña,12965.0,504.0,1019.0,156231.0,322.599228,652.239312,87.0
Huelva (capital),11493.0,381.0,868.0,143837.0,264.883166,603.460862,87.0
Sierra de Huelva-Andévalo Central,3675.0,106.0,280.0,67391.0,157.291033,415.485747,24.0
Moguer,1799.0,91.0,180.0,21867.0,416.152193,823.158184,19.0
Isla Cristina,2780.0,79.0,186.0,21393.0,369.279671,869.443276,18.0
Bollullos Par del Condado,1173.0,51.0,75.0,14387.0,354.486689,521.303955,16.0
Lepe,2698.0,98.0,212.0,27880.0,351.506456,760.401722,14.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Sanlúcar de Guadiana,27.0,15.0,20.0,403.0,3722.084367,4962.779156,2.0
Cumbres Mayores,120.0,6.0,41.0,1749.0,343.053173,2344.196684,2.0
Cañaveral de León,15.0,4.0,8.0,396.0,1010.101010,2020.202020,2.0
Marines (Los),14.0,6.0,8.0,399.0,1503.759398,2005.012531,0.0
Nava (La),6.0,2.0,4.0,258.0,775.193798,1550.387597,0.0
Escacena del Campo,197.0,21.0,34.0,2287.0,918.233494,1486.663752,3.0
Corteconcepción,39.0,4.0,7.0,536.0,746.268657,1305.970149,0.0
Lucena del Puerto,336.0,21.0,37.0,3261.0,643.974241,1134.621282,2.0
Villarrasa,209.0,21.0,24.0,2211.0,949.796472,1085.481682,3.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Cumbres Mayores,120.0,6.0,41.0,1749.0,343.053173,2344.196684,2.0,0.146341
Villanueva de los Castillejos,190.0,2.0,12.0,2825.0,70.796460,424.778761,1.0,0.166667
Encinasola,161.0,2.0,11.0,1310.0,152.671756,839.694656,1.0,0.181818
Cerro de Andévalo (El),185.0,2.0,9.0,2327.0,85.947572,386.764074,0.0,0.222222
Aracena,624.0,6.0,26.0,8255.0,72.683222,314.960630,3.0,0.230769
Calañas,239.0,4.0,17.0,2768.0,144.508671,614.161850,1.0,0.235294
Santa Ana la Real,22.0,1.0,4.0,475.0,210.526316,842.105263,1.0,0.250000
Punta Umbría,1162.0,35.0,126.0,15355.0,227.938782,820.579616,4.0,0.277778
Puebla de Guzmán,163.0,2.0,7.0,3092.0,64.683053,226.390686,0.0,0.285714
